# Selection Diagrams

This notebook is for creating diagrams of how selection works, based on *real* data from the yt sample datasets.

We'll construct and highlight diagrams showing selection of:

 * Slices
 * Regions
 * Spheres
 
This should be straightforward since we can use the built-in yt selection routines.

In [ ]:
import drawSvg as draw
import yt
import numpy as np
import ipywidgets

In [ ]:
yt.utilities.logger.disable_stream_logging()

In [ ]:
#ds = yt.load("~/data/IsolatedGalaxy/galaxy0030/galaxy0030")
#ds = yt.load("/home/matthewturk/data/EnzoKelvinHelmholtz/DD0011/DD0011")
#ds = yt.load("/home/matthewturk/data/PopIII_mini/DD0034/DD0034")
ds = yt.load("~/data/enzo_tiny_cosmology/DD0046/DD0046")

In [ ]:
s = ds.sphere("c", 0.5)

In [ ]:
xax = 0
yax = 1
margin = 20
width = height = 500

In [ ]:
def make_image_data(source):
    selected_grids = source.selector.select_grids(
        source.ds.index.grid_left_edge,
        source.ds.index.grid_right_edge,
        source.ds.index.grid_levels
    )
    frb = source.integrate("density", axis="z").to_frb(1.0, (512, 512))
    frb_data = np.log10(frb["density"][::-1,:].v)
    frb_bitmap = yt.write_image(frb_data, None)
    frb_bitmap[frb_data == -np.inf,3] = 0
    image_data = yt.write_bitmap(frb_bitmap, None)
    
    im = draw.Image(data = image_data, x = margin, y = margin, width = width - 2*margin,
                height = height - 2*margin, mimeType="image/png", opacity = 1)
    return im, selected_grids

In [ ]:
def make_svg(source, selected = 'black', unselected = 'grey', show_image = True):
    c = draw.Drawing(width = width, height = height)

    im, sg = make_image_data(source)

    if show_image:
        c.append(im)

    scale_x = lambda v: margin + (width - 2*margin) * v
    scale_y = lambda v: margin + (height - 2*margin) * v
    domain = draw.Rectangle(scale_x(0), scale_y(0),
                            scale_x(1.0) - scale_x(0.0),
                            scale_y(1.0) - scale_y(0.0),
                            fill = 'none', stroke_width=2.0,
                            stroke = 'black')
    c.append(domain)
    def draw_grid(stroke, stroke_width, LE, RE):
        grid_outline = draw.Rectangle(
            scale_x(LE[xax]),
            scale_y(LE[yax]),
            scale_x(RE[xax]) - scale_x(LE[xax]),
            scale_y(RE[yax]) - scale_y(LE[yax]),
            fill = '#AAAAAA', fill_opacity=0.0, stroke = stroke, stroke_width=stroke_width)
        return grid_outline
    [c.append(draw_grid(unselected, 0.5, grid.LeftEdge.v, grid.RightEdge.v)) for grid in ds.index.grids[~sg]]
    [c.append(draw_grid(selected, 1.0, grid.LeftEdge.v, grid.RightEdge.v)) for grid in ds.index.grids[sg]]
    return c

In [ ]:
cv = (0.0, 1.0, 0.01)

In [ ]:
@ipywidgets.interact(left_edge_x = cv, left_edge_y = cv, left_edge_z = cv,
                     right_edge_x = cv, right_edge_y = cv, right_edge_z = cv)
def make_reg(left_edge_x = 0.0, left_edge_y = 0.0, left_edge_z = 0.0,
             right_edge_x = 1.0, right_edge_y = 1.0, right_edge_z = 1.0):
    r = ds.r[left_edge_x:right_edge_x, left_edge_y:right_edge_y, left_edge_z:right_edge_z]
    return make_svg(r, 'black', 'grey')

In [ ]:
@ipywidgets.interact(center_x = (0.0, 1.0, 0.01),
                     center_y = (0.0, 1.0, 0.01),
                     center_z = (0.0, 1.0, 0.01),
                     radius = (0.0, 1.0, 0.01),
                    show_image = True)
def make_sphere(center_x = 0.5, center_y = 0.5, center_z = 0.5,
             radius = 0.15, show_image = True):
    c = make_svg(ds.sphere((center_x, center_y, center_z), radius), 'black', 'grey', show_image = show_image)
    return c